# Chapter 3: Silver Merge

**Purpose:** Merge all Bronze-stage datasets onto a temporal spine (`entity_id` x `as_of_date`), producing a unified Silver feature matrix aligned on the SnapshotGrid.

**When to use this notebook:**
- Notebooks 00–01d have run and produced Bronze datasets in the RunNamespace
- A SnapshotGrid has been defined (notebook 01 or 01c)
- You want to create a single wide table aligned on `(entity_id, as_of_date)` for downstream analysis

**What you'll learn:**
- How to build a temporal spine from the SnapshotGrid
- How event datasets equi-join on `(entity_id, as_of_date)`
- How entity datasets broadcast across all observation dates
- How to validate temporal integrity of the merged result

**Outputs:**
- Silver merged Delta table at `namespace.silver_merged_path`
- Merge report (row counts, column counts, conflict resolutions)
- Temporal integrity validation

---

## Spine Table Pattern

```
SnapshotGrid.grid_dates    All entity_ids from Bronze datasets
        |                              |
        v                              v
   [2024-01-01,          x       [A, B, C, ...]
    2024-02-01,                         |
    2024-03-01, ...]                    |
        |                              |
        +--------- CROSS PRODUCT ------+
                       |
                       v
              SPINE (entity_id, as_of_date)
                       |
        +--------------+--------------+
        |              |              |
   LEFT JOIN      LEFT JOIN      LEFT JOIN
   (equi-join)    (broadcast)    (as-of)
        |              |              |
   Event Bronze   Entity Bronze  Entity w/ timestamp
   (has as_of)   (no as_of)     (feature_timestamp)
        |              |              |
        +--------> SILVER MERGED <----+
```

## 3.1 Setup

In [ ]:
from customer_retention.analysis.notebook_progress import track_and_export_previous
from customer_retention.analysis.visualization import display_table

track_and_export_previous("03_dataset_merge.ipynb")

from customer_retention.analysis.auto_explorer import mark_notebook
from customer_retention.analysis.auto_explorer.active_dataset_store import (
    load_active_dataset,
    load_merge_dataset,
    load_merge_dataset_distributed,
)
from customer_retention.analysis.auto_explorer.project_context import ProjectContext
from customer_retention.analysis.auto_explorer.run_namespace import RunNamespace
from customer_retention.analysis.auto_explorer.snapshot_grid import SnapshotGrid
from customer_retention.core.compat import (
    concat,
    is_remote_spark,
    is_spark_available,
    native_pd,
    pd,
    safe_describe,
    safe_memory_usage_bytes,
)
from customer_retention.core.config.column_config import DatasetGranularity
from customer_retention.core.config.experiments import (
    FINDINGS_DIR,
)
from customer_retention.integrations.adapters.factory import get_delta
from customer_retention.stages.temporal import (
    DatasetMergeInput,
    MergeConfig,
    SparkTemporalMerger,
    TemporalMerger,
)

## 3.2 Load Context

In [ ]:
_namespace = RunNamespace.from_env() or RunNamespace.from_latest()
if _namespace is None:
    raise RuntimeError("No RunNamespace found. Run notebook 00 first.")
mark_notebook(_namespace, "03_dataset_merge.ipynb")

context = ProjectContext.load(_namespace.project_context_path)
grid = SnapshotGrid.load(_namespace.snapshot_grid_path)

print(f"Run: {_namespace.run_id}")
print(f"Project: {context.project_name}")
print(f"Datasets registered: {len(context.datasets)}")
print(f"Grid dates: {len(grid.grid_dates)} ({grid.cadence_interval.value} cadence)")
if grid.grid_dates:
    print(f"  Range: {grid.grid_dates[0]} to {grid.grid_dates[-1]}")

## 3.3 Load Bronze Datasets

In [ ]:
dataset_names = _namespace.list_datasets()
merge_inputs: list[DatasetMergeInput] = []
entity_id_parts: list = []

_use_distributed = is_spark_available() and not is_remote_spark()

print("=" * 70)
print("BRONZE DATASETS")
print("=" * 70 + "\n")

for name in dataset_names:
    entry = context.datasets.get(name)
    granularity = entry.granularity if entry else DatasetGranularity.UNKNOWN
    df = load_merge_dataset_distributed(_namespace, name, granularity) if _use_distributed else load_merge_dataset(_namespace, name, granularity)
    entity_col = entry.entity_column if entry else None
    feature_ts = entry.time_column if (entry and granularity == DatasetGranularity.ENTITY_LEVEL and entry.time_column) else None

    if entity_col and entity_col in df.columns:
        entity_id_parts.append(df[entity_col])
        if entity_col != "entity_id":
            df = df.rename(columns={entity_col: "entity_id"})

    merge_inputs.append(DatasetMergeInput(
        name=name,
        df=df,
        granularity=granularity,
        feature_timestamp_column=feature_ts,
    ))

    label = granularity.value if granularity else "unknown"
    print(f"  {name}")
    print(f"    Granularity: {label}")
    print(f"    Shape: {df.shape[0]:,} rows x {df.shape[1]} cols")
    if entity_col and entity_col != "entity_id":
        print(f"    Entity column: {entity_col} -> entity_id")
    if feature_ts:
        print(f"    Feature timestamp: {feature_ts}")
    print()

all_entity_ids = concat(entity_id_parts, ignore_index=True) if entity_id_parts else native_pd.Series(dtype=str)
print(f"Total datasets loaded: {len(merge_inputs)}")

## 3.4 Build Spine

In [ ]:
config = MergeConfig(entity_key="entity_id")
merger = SparkTemporalMerger(config=config) if _use_distributed else TemporalMerger(config=config)

spine = merger.build_spine(all_entity_ids, grid.grid_dates)

print("=" * 70)
print("SPINE")
print("=" * 70)
print(f"  Unique entities: {spine['entity_id'].nunique():,}")
print(f"  Grid dates:      {spine['as_of_date'].nunique()}")
print(f"  Total rows:      {len(spine):,}")
_spine_bytes = safe_memory_usage_bytes(spine)
if _spine_bytes:
    print(f"  Estimated size:  {_spine_bytes / 1024 / 1024:.1f} MB (spine only)")

## 3.5 Merge

In [ ]:
merged, report = merger.merge_all(spine, merge_inputs)

print("=" * 70)
print("MERGE RESULTS")
print("=" * 70)
print(f"  Datasets merged:  {len(report.datasets_merged)}")
print(f"  Final shape:      {merged.shape[0]:,} rows x {merged.shape[1]} cols")
print()

print("Per-dataset column counts:")
for ds_name, n_cols in report.columns_per_dataset.items():
    print(f"  {ds_name}: +{n_cols} columns")

if report.renamed_columns:
    print(f"\nColumn conflicts resolved: {len(report.renamed_columns)}")
    for original, renamed in list(report.renamed_columns.items())[:10]:
        print(f"  {original}")

## 3.6 Validation

In [ ]:
print("=" * 70)
print("TEMPORAL INTEGRITY")
print("=" * 70)

ti = report.temporal_integrity
if ti:
    status = "PASS" if ti.get("valid", True) else "FAIL"
    print(f"  Status: {status}")
    issues = ti.get("issues", [])
    if issues:
        for issue in issues:
            print(f"  Issue: {issue.get('type', 'unknown')} - {issue.get('message', '')}")
    else:
        print("  No temporal integrity issues detected.")
else:
    print("  Temporal validation was skipped.")

# Check spine preservation
assert len(merged) == report.spine_rows, (
    f"Row count mismatch: merged={len(merged)}, spine={report.spine_rows}"
)
print(f"\n  Spine row count preserved: {len(merged):,}")

## 3.7 Save Merged Dataset

In [ ]:
output_path = _namespace.silver_merged_path
delta = get_delta() if _use_distributed else get_delta(force_local=True)

print(f"Writing to: {output_path}")
print(f"  Adapter: {type(delta).__name__}")
print(f"  DataFrame: {type(merged).__name__} {merged.shape}")
_merged_bytes = safe_memory_usage_bytes(merged)
if _merged_bytes:
    print(f"  Memory: {_merged_bytes / 1024**2:.0f} MB")

delta.write(merged, str(output_path), mode="overwrite")

print(f"Silver merged dataset saved to: {output_path}")
print(f"  Shape: {report.spine_rows:,} rows x {report.total_columns} cols")

from customer_retention.analysis.auto_explorer.findings import ExplorationFindings

all_findings_paths = _namespace.discover_all_findings(prefer_aggregated=True)
dataset_findings = [ExplorationFindings.load(str(p)) for p in all_findings_paths]
merged_findings = ExplorationFindings.merge_from_datasets(
    dataset_findings,
    row_count=report.spine_rows,
    column_count=report.total_columns,
    source_path=str(output_path),
    renamed_columns=report.renamed_columns,
    entity_key=config.entity_key,
)
_namespace.merged_dir.mkdir(parents=True, exist_ok=True)
merged_findings.save(str(_namespace.merged_findings_path))

print(f"  Merged findings saved to: {_namespace.merged_findings_path}")
print(f"  Columns catalogued: {len(merged_findings.columns)}")

## 3.8 Preview

In [ ]:
print("First 10 rows:")
display_table(merged.head(10))

print("\nDescriptive statistics:")
display_table(safe_describe(merged))

## 3.9 Summary

In [ ]:
print("=" * 70)
print("SILVER MERGE SUMMARY")
print("=" * 70)
print(f"  Spine: {report.spine_entities:,} entities x {report.spine_dates} dates = {report.spine_rows:,} rows")
print(f"  Datasets merged: {', '.join(report.datasets_merged)}")
print(f"  Total columns: {report.total_columns}")
if report.renamed_columns:
    print(f"  Renamed columns: {len(report.renamed_columns)}")
print(f"  Output: {output_path}")
print()
print("Next steps:")
print("  04_column_deep_dive.ipynb  - Feature deep dive on merged data")
print("  05_relationship_analysis.ipynb - Correlation and interaction analysis")

---

## Summary: What We Learned

In this notebook, we:

1. **Loaded Context** - RunNamespace, ProjectContext, and SnapshotGrid
2. **Loaded Bronze Datasets** - All datasets from the namespace with their granularity metadata
3. **Built Spine** - Cross product of all entity IDs and grid dates
4. **Merged** - Left-joined all datasets onto the spine (event equi-join, entity broadcast)
5. **Validated** - Checked temporal integrity of the merged result
6. **Saved** - Silver merged Delta table for downstream notebooks

---

## Next Steps

Continue to **04_column_deep_dive.ipynb** to:
- Deep dive into individual columns of the merged dataset
- Analyze distributions, outliers, and missing patterns

Or **05_relationship_analysis.ipynb** to:
- Explore correlations between features on the merged data
- Analyze feature-target relationships
- Detect multicollinearity

> **Save Reminder:** Save this notebook (Ctrl+S / Cmd+S) before running the next one.
> The next notebook will automatically export this notebook's HTML documentation from the saved file.